In [ ]:
# 필요한 라이브러리 설치
!pip install ultralytics roboflow scikit-learn pandas iterative-stratification

import os
from collections import defaultdict, Counter
import yaml
import shutil
import cv2
from tqdm import tqdm
from ultralytics import YOLO
from roboflow import Roboflow
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.preprocessing import MultiLabelBinarizer

# 저장 폴더 설정
MODEL_SAVE_DIR = "custom_model_weights"
CROPPED_WEIGHTS_DIR = os.path.join(MODEL_SAVE_DIR, "cropped_training")
SPECIFIC_WEIGHTS_DIR = os.path.join(MODEL_SAVE_DIR, "specific_class_training")
REFINEMENT_WEIGHTS_DIR = os.path.join(MODEL_SAVE_DIR, "refinement_training")

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(CROPPED_WEIGHTS_DIR, exist_ok=True)
os.makedirs(SPECIFIC_WEIGHTS_DIR, exist_ok=True)
os.makedirs(REFINEMENT_WEIGHTS_DIR, exist_ok=True)

# 라벨 파일 필터링 및 재매핑 함수
def filter_and_remap_labels(label_dir, valid_classes):
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        filtered_lines = []

        with open(label_path, 'r') as file:
            for line in file:
                parts = line.strip().split()
                class_id = int(parts[0])
                if class_id in valid_classes:
                    # 클래스 ID 재매핑
                    new_class_id = valid_classes.index(class_id)
                    parts[0] = str(new_class_id)
                    filtered_line = ' '.join(parts) + '\n'
                    filtered_lines.append(filtered_line)

        if filtered_lines:
            with open(label_path, 'w') as file:
                file.writelines(filtered_lines)
        else:
            # 유효한 클래스가 없으면 라벨 파일과 이미지 파일 삭제
            os.remove(label_path)
            image_path = os.path.join(label_dir.replace("labels", "images"), label_file.replace(".txt", ".jpg"))
            if os.path.exists(image_path):
                os.remove(image_path)

# data.yaml 파일 업데이트 함수
def update_yaml(yaml_path, class_names):
    with open(yaml_path, 'r') as yaml_file:
        data = yaml.safe_load(yaml_file)

    data['nc'] = len(class_names)
    data['names'] = class_names

    with open(yaml_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False)

# 이미지와 라벨들을 하나의 폴더로 합치는 함수
def combine_datasets(dataset_path, combined_path):
    images_combined_dir = os.path.join(combined_path, "images")
    labels_combined_dir = os.path.join(combined_path, "labels")
    os.makedirs(images_combined_dir, exist_ok=True)
    os.makedirs(labels_combined_dir, exist_ok=True)

    for subset in ["train", "valid", "test"]:
        images_dir = os.path.join(dataset_path, subset, "images")
        labels_dir = os.path.join(dataset_path, subset, "labels")

        if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
            continue

        for image_file in os.listdir(images_dir):
            src_image_path = os.path.join(images_dir, image_file)
            dst_image_path = os.path.join(images_combined_dir, image_file)
            shutil.copy(src_image_path, dst_image_path)

        for label_file in os.listdir(labels_dir):
            src_label_path = os.path.join(labels_dir, label_file)
            dst_label_path = os.path.join(labels_combined_dir, label_file)
            shutil.copy(src_label_path, dst_label_path)

# 전체 데이터셋을 클래스별로 균등하게 분할하는 함수 (70:20:10 비율)
def split_dataset_evenly(combined_path, output_path, split_ratios=(0.7, 0.2, 0.1)):
    images_dir = os.path.join(combined_path, "images")
    labels_dir = os.path.join(combined_path, "labels")

    image_label_pairs = []
    class_image_counts = defaultdict(int)
    class_indices = defaultdict(list)

    # 모든 레이블 파일을 읽고, 각 이미지의 클래스 목록을 생성
    for idx, label_file in enumerate(os.listdir(labels_dir)):
        label_path = os.path.join(labels_dir, label_file)
        image_name = label_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_dir, image_name)

        if not os.path.exists(image_path):
            continue

        with open(label_path, 'r') as f:
            lines = f.readlines()

        classes_in_image = set()
        for line in lines:
            class_id = int(line.strip().split()[0])
            classes_in_image.add(class_id)
            class_image_counts[class_id] += 1

        if classes_in_image:
            image_label_pairs.append({
                'image_path': image_path,
                'label_path': label_path,
                'classes': list(classes_in_image),
                'image_name': image_name,
                'label_name': label_file
            })
            # 클래스별 인덱스 저장
            for class_id in classes_in_image:
                class_indices[class_id].append(idx)

    if not image_label_pairs:
        print("데이터셋이 비어 있습니다.")
        return

    df = pd.DataFrame(image_label_pairs)

    mlb = MultiLabelBinarizer(classes=valid_classes)
    Y = mlb.fit_transform(df['classes'])

    from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=split_ratios[2], random_state=42)
    train_val_indices, test_indices = next(msss.split(df, Y))

    Y_train_val = Y[train_val_indices]
    Y_test = Y[test_indices]

    msss_val = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=split_ratios[1]/(split_ratios[0] + split_ratios[1]), random_state=42)
    train_indices, valid_indices = next(msss_val.split(df.iloc[train_val_indices], Y_train_val))

    train_indices = train_val_indices[train_indices]
    valid_indices = train_val_indices[valid_indices]

    datasets = {
        'train': df.iloc[train_indices],
        'valid': df.iloc[valid_indices],
        'test': df.iloc[test_indices]
    }

    for subset in ["train", "valid", "test"]:
        subset_images_dir = os.path.join(output_path, subset, 'images')
        subset_labels_dir = os.path.join(output_path, subset, 'labels')
        if os.path.exists(subset_images_dir):
            shutil.rmtree(subset_images_dir)
        if os.path.exists(subset_labels_dir):
            shutil.rmtree(subset_labels_dir)

    for subset, df_subset in datasets.items():
        images_output_dir = os.path.join(output_path, subset, 'images')
        labels_output_dir = os.path.join(output_path, subset, 'labels')
        os.makedirs(images_output_dir, exist_ok=True)
        os.makedirs(labels_output_dir, exist_ok=True)

        for _, row in df_subset.iterrows():
            image_src_path = row['image_path']
            image_dst_path = os.path.join(images_output_dir, row['image_name'])
            shutil.copy(image_src_path, image_dst_path)

            label_src_path = row['label_path']
            label_dst_path = os.path.join(labels_output_dir, row['label_name'])
            shutil.copy(label_src_path, label_dst_path)

    print("데이터셋 분할 완료: Train, Validation, Test")

def crop_and_save_objects(dataset_path, output_path, valid_classes):
    for subset in ["train", "valid", "test"]:
        labels_path = os.path.join(dataset_path, subset, "labels")
        images_path = os.path.join(dataset_path, subset, "images")

        cropped_images_path = os.path.join(output_path, subset, "images")
        cropped_labels_path = os.path.join(output_path, subset, "labels")
        os.makedirs(cropped_images_path, exist_ok=True)
        os.makedirs(cropped_labels_path, exist_ok=True)

        for label_file in tqdm(os.listdir(labels_path), desc=f"Cropping {subset}"):
            label_path = os.path.join(labels_path, label_file)
            image_path = os.path.join(images_path, label_file.replace(".txt", ".jpg"))
            if not os.path.exists(image_path):
                continue

            with open(label_path, "r") as file:
                lines = file.readlines()

            image = cv2.imread(image_path)
            if image is None:
                continue
            height, width, _ = image.shape

            for idx, line in enumerate(lines):
                class_id, x_center, y_center, w, h = map(float, line.strip().split())
                if int(class_id) not in valid_classes:
                    continue
                new_class_id = int(class_id)

                x_min = int((x_center - w / 2) * width)
                y_min = int((y_center - h / 2) * height)
                x_max = int((x_center + w / 2) * width)
                y_max = int((y_center + h / 2) * height)

                x_min = max(0, x_min)
                y_min = max(0, y_min)
                x_max = min(width, x_max)
                y_max = min(height, y_max)

                cropped_image = image[y_min:y_max, x_min:x_max]
                if cropped_image.size == 0:
                    continue

                cropped_image_name = f"{os.path.splitext(label_file)[0]}_{idx}.jpg"
                cropped_label_name = f"{os.path.splitext(label_file)[0]}_{idx}.txt"
                cropped_image_path = os.path.join(cropped_images_path, cropped_image_name)
                cropped_label_path = os.path.join(cropped_labels_path, cropped_label_name)

                cv2.imwrite(cropped_image_path, cropped_image)
                with open(cropped_label_path, "w") as cropped_label_file:
                    cropped_label_file.write(f"{new_class_id} 0.5 0.5 1.0 1.0\n")

def create_data_yaml(output_path, class_names, fixed_test_path=None):
    data_yaml = {
        'path': output_path,
        'train': os.path.join(output_path, 'train', 'images'),
        'val': os.path.join(output_path, 'valid', 'images'),
        'test': os.path.join(output_path, 'test', 'images'),
        'nc': len(class_names),
        'names': class_names
    }
    yaml_path = os.path.join(output_path, 'data.yaml')
    with open(yaml_path, 'w') as yaml_file:
        yaml.dump(data_yaml, yaml_file, default_flow_style=False)
    return yaml_path

def compute_class_weights_log(label_dir, num_classes):
    class_counts = Counter()
    for label_file in os.listdir(label_dir):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f:
                class_id = int(line.strip().split()[0])
                class_counts[class_id] += 1
    total = sum(class_counts.values())
    weights = [np.log(1 + total / class_counts[i]) if i in class_counts else 0 for i in range(num_classes)]
    return weights

rf = Roboflow(api_key="68TRJG3oL0xut9Ks0kkD")
project = rf.workspace("ship-xallx").project("test-au9yp")
version = project.version(5)
dataset = version.download("yolov8")

dataset_path = dataset.location
data_yaml_path = os.path.join(dataset.location, "data.yaml")
combined_dataset_path = os.path.join(dataset.location, "combined_dataset")
cropped_dataset_path = os.path.join(dataset.location, "cropped_dataset")
os.makedirs(combined_dataset_path, exist_ok=True)
os.makedirs(cropped_dataset_path, exist_ok=True)

valid_classes = list(range(10))
with open(data_yaml_path, "r") as yaml_file:
    data_config = yaml.safe_load(yaml_file)
all_class_names = data_config["names"]
class_names = [all_class_names[i] for i in valid_classes]

print("라벨 파일 필터링 및 재매핑 중...")
for subset in ["train", "valid", "test"]:
    label_dir = os.path.join(dataset_path, subset, "labels")
    filter_and_remap_labels(label_dir, valid_classes)

print("data.yaml 파일 업데이트 중...")
update_yaml(data_yaml_path, class_names)

print("이미지와 라벨들을 하나의 폴더로 합치는 중...")
combine_datasets(dataset_path, combined_dataset_path)

print("데이터셋을 70:20:10 비율로 분할하는 중...")
split_dataset_evenly(combined_dataset_path, dataset_path, split_ratios=(0.7, 0.2, 0.1))

print("새로운 data.yaml 파일 생성 중...")
data_yaml_path = create_data_yaml(dataset_path, class_names)

print("데이터셋에서 객체를 크롭 중...")
crop_and_save_objects(dataset_path, cropped_dataset_path, valid_classes)

cropped_data_yaml_path = create_data_yaml(cropped_dataset_path, class_names)

num_classes = len(class_names)
print(f"데이터셋의 클래스 수: {num_classes}")

label_dir = os.path.join(cropped_dataset_path, "train", "labels")
class_weights = compute_class_weights_log(label_dir, num_classes=num_classes)
print("클래스 가중치:", class_weights)

cropped_project_dir = CROPPED_WEIGHTS_DIR
model = YOLO("yolo11l.pt")
print("크롭된 데이터셋으로 학습 중...")
model.train(
    data=cropped_data_yaml_path,
    epochs=70,
    imgsz=640,
    batch=16,
    name="cropped_objects_training",
    patience=5,
    lr0=0.0001,
    optimizer='AdamW',
    dropout=0.2,
    freeze=[0],
    augment= False,
    project=cropped_project_dir
)

cropped_best_weights_path = os.path.join(cropped_project_dir, "cropped_objects_training", "weights", "best.pt")
if os.path.exists(cropped_best_weights_path):
    model_save_path = os.path.join(MODEL_SAVE_DIR, "yolov8_cropped_best.pt")
    shutil.copy(cropped_best_weights_path, model_save_path)
    print(f"크롭된 데이터셋의 최적 가중치가 '{model_save_path}'에 저장되었습니다.")
else:
    print("크롭된 최적 가중치를 찾을 수 없습니다. 학습 과정을 확인하세요.")

refinement_project_dir = REFINEMENT_WEIGHTS_DIR
print("원본 이미지로 재학습 중...")
model = YOLO(cropped_best_weights_path)
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    name="refinement_training",
    patience=7,
    lr0=0.0001,
    optimizer='AdamW',
    freeze=[0],
    dropout=0.2,
    augment= False,
    project=refinement_project_dir
)

refined_best_weights_path = os.path.join(refinement_project_dir, "refinement_training", "weights", "best.pt")
if os.path.exists(refined_best_weights_path):
    model_save_path = os.path.join(MODEL_SAVE_DIR, "yolov8_refined_best.pt")
    shutil.copy(refined_best_weights_path, model_save_path)
    print(f"재학습된 최적 가중치가 '{model_save_path}'에 저장되었습니다.")
else:
    print("재학습된 최적 가중치를 찾을 수 없습니다. 학습 과정을 확인하세요.")

print("모델 평가 중...")
# 최종 성능은 test 세트로 평가
metrics_test = model.val(data=data_yaml_path, split='test')
results_test = metrics_test.results_dict
map50 = results_test.get('metrics/mAP50(B)', None)
map50_95 = results_test.get('metrics/mAP50-95(B)', None)
precision = results_test.get('metrics/precision(B)', None)
recall = results_test.get('metrics/recall(B)', None)

print(f"Test set mAP50: {map50:.4f}" if map50 is not None else "mAP50를 찾을 수 없습니다.")
print(f"Test set mAP50-95: {map50_95:.4f}" if map50_95 is not None else "mAP50-95를 찾을 수 없습니다.")
print(f"Test set 정밀도: {precision:.4f}" if precision is not None else "정밀도를 찾을 수 없습니다.")
print(f"Test set 재현율: {recall:.4f}" if recall is not None else "재현율을 찾을 수 없습니다.")

print("\n클래스별 성능 (Test):")
class_names_mapping = metrics_test.names
for class_id, class_name in class_names_mapping.items():
    try:
        class_stats = metrics_test.class_result(class_id)
        precision_cls, recall_cls, mAP50_cls, mAP50_95_cls = class_stats
        print(f"클래스 {class_id} ({class_name}):")
        print(f"  정밀도: {precision_cls:.4f}")
        print(f"  재현율: {recall_cls:.4f}")
        print(f"  mAP@50: {mAP50_cls:.4f}")
        print(f"  mAP@50-95: {mAP50_95_cls:.4f}")
    except IndexError:
        print(f"클래스 {class_id} ({class_name}): 데이터가 없습니다.")

print("\n모델 평가가 완료되었습니다.")

# ------------------- 과적합 여부 확인 추가 -------------------
print("\n# 과적합 여부 확인")

# 훈련 데이터셋에 대한 성능 평가
metrics_train = model.val(data=data_yaml_path, split='train')
train_results = metrics_train.results_dict
train_map50 = train_results.get('metrics/mAP50(B)', None)

# 검증 데이터셋에 대한 성능 평가
metrics_val = model.val(data=data_yaml_path, split='val')
val_results = metrics_val.results_dict
val_map50 = val_results.get('metrics/mAP50(B)', None)

# 테스트 데이터셋 성능은 이미 위에서 평가함
test_map50 = map50

if train_map50 is not None and val_map50 is not None:
    print(f"Train set mAP50: {train_map50:.4f}")
    print(f"Val set mAP50: {val_map50:.4f}")
    print(f"Test set mAP50: {test_map50:.4f}" if test_map50 is not None else "Test mAP50 없음")

    # 과적합 판단 기준 예시: 훈련 mAP50이 검증 mAP50보다 지나치게 높고, 검증 mAP50 대비 테스트 mAP50 저하가 있다면 과적합 가능성
    if train_map50 > val_map50 + 0.05:
        print("과적합 의심: 훈련 성능이 검증 성능보다 크게 높습니다.")
    else:
        print("과적합 징후가 뚜렷하지 않습니다.")
else:
    print("Train 또는 Val 평가를 수행할 수 없어 과적합 여부를 판단하기 어렵습니다.")

# ------------------- 끝 -------------------


In [ ]:
!pip install ultralytics roboflow

import torch
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from roboflow import Roboflow
rf = Roboflow(api_key="68TRJG3oL0xut9Ks0kkD")
project = rf.workspace("ship-xallx").project("test-au9yp")
version = project.version(7)
dataset = version.download("yolov8")

# 1. YOLO 모델 불러오기 (YOLOv11 기준)
# 모델 파일 경로
MODEL_PATH = '/content/ver5_yolov11_refined_best.pt'  # 사전에 학습된 YOLOv11 가중치 파일 경로

# 모델 로드
from ultralytics import YOLO  # Ultralytics 라이브러리 사용
model = YOLO(MODEL_PATH)  # YOLOv11 모델 로드

# 2. 테스트 데이터셋 설정
TEST_DATASET_PATH = '/content/test-7/test/images'  # 테스트 이미지가 저장된 디렉토리
OUTPUT_PATH = './output_images/'  # 결과 이미지 저장 디렉토리
Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

# 3. 이미지 평가 함수 정의
def detect_and_plot(image_path, model):
    """이미지를 YOLO 모델로 탐지하고 결과를 출력합니다."""
    # 이미지 읽기
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 고유한 결과 디렉토리 생성
    image_name = Path(image_path).stem  # 이미지 이름 추출
    unique_output_dir = Path(OUTPUT_PATH) / f'results_{image_name}'
    unique_output_dir.mkdir(parents=True, exist_ok=True)

    # 모델 추론
    results = model.predict(source=img_rgb, save=True, project=str(unique_output_dir), name='results')

    # 결과 이미지 파일 경로 가져오기
    output_images = list(unique_output_dir.glob('results/*.jpg'))  # 결과 이미지 찾기

    if output_images:
        output_image_path = str(output_images[0])  # 첫 번째 결과 이미지 선택
    else:
        print(f"No output file found for {image_path}")
        return

    # 결과 이미지 읽기 및 표시
    output_img = cv2.imread(output_image_path)
    if output_img is not None:
        plt.imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
    else:
        print(f"Failed to read the result image: {output_image_path}")

# 4. 테스트 데이터셋 평가 및 결과 출력
image_paths = list(Path(TEST_DATASET_PATH).glob('*.*'))  # 이미지 파일 검색

for image_path in image_paths:
    print(f"Processing {image_path}...")
    detect_and_plot(str(image_path), model)

print("모든 이미지 평가 및 출력 완료.")



In [ ]:
# Macro F1 Score 계산을 위한 수정된 코드 셀

# 필요한 라이브러리 임포트
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm
from collections import defaultdict
import yaml

# IoU 계산 함수
def compute_iou(box1, box2):
    """
    두 박스 간의 Intersection over Union (IoU)을 계산합니다.
    박스는 [x1, y1, x2, y2] 형식이어야 합니다.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = max(0, (box1[2]-box1[0])) * max(0, (box1[3]-box1[1]))
    box2_area = max(0, (box2[2]-box2[0])) * max(0, (box2[3]-box2[1]))

    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0
    else:
        return inter_area / union_area

# 예측과 실제 라벨을 매칭하고 TP, FP, FN을 계산하는 함수
def match_predictions(preds, targets, iou_threshold=0.5):
    """
    예측(preds)과 실제 객체(targets)를 매칭하여 클래스별 TP, FP, FN을 계산합니다.
    """
    TP = defaultdict(int)
    FP = defaultdict(int)
    FN = defaultdict(int)

    matched_targets = set()

    for pred in preds:
        pred_bbox = pred['bbox']
        pred_class = pred['class']
        best_iou = 0
        best_target = None
        for idx, target in enumerate(targets):
            if target['class'] != pred_class or idx in matched_targets:
                continue
            iou = compute_iou(pred_bbox, target['bbox'])
            if iou > best_iou:
                best_iou = iou
                best_target = idx
        if best_iou >= iou_threshold:
            TP[pred_class] += 1
            matched_targets.add(best_target)
        else:
            FP[pred_class] += 1

    for idx, target in enumerate(targets):
        if idx not in matched_targets:
            FN[target['class']] += 1

    return TP, FP, FN

# 데이터셋 및 모델 경로 설정
MODEL_SAVE_DIR = "custom_model_weights"
trained_weights_path = "/content/custom_model_weights/ver5_yolov11_refined_best.pt"  # 훈련된 모델 가중치 경로

# 테스트 데이터셋 경로 (사용 중인 데이터셋 경로에 맞게 수정)
dataset_path = "/content/test-5"  # 예: "/content/dataset"
test_images_dir = os.path.join(dataset_path, "test", "images")
test_labels_dir = os.path.join(dataset_path, "test", "labels")
data_yaml_path = os.path.join(dataset_path, "data.yaml")

# 클래스 이름 로드
with open(data_yaml_path, "r") as yaml_file:
    data_config = yaml.safe_load(yaml_file)
class_names = data_config["names"]

# 모델 로드
model = YOLO(trained_weights_path)

# 메트릭 초기화
metrics = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})

# 테스트 이미지 파일 리스트
test_image_files = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# 모든 테스트 이미지에 대해 예측 및 메트릭 계산
for image_file in tqdm(test_image_files, desc="테스트 이미지 처리 중"):
    image_path = os.path.join(test_images_dir, image_file)
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(test_labels_dir, label_file)

    # 실제 객체 로드
    targets = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                print(f"라벨 파일 형식 오류: {label_path}")
                continue
            class_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:5])
            img = cv2.imread(image_path)
            if img is None:
                print(f"이미지 로드 실패: {image_path}")
                continue
            height, width, _ = img.shape
            x1 = max(0, (x_center - w/2) * width)
            y1 = max(0, (y_center - h/2) * height)
            x2 = min(width, (x_center + w/2) * width)
            y2 = min(height, (y_center + h/2) * height)
            targets.append({
                'bbox': [x1, y1, x2, y2],
                'class': class_id
            })

    # 예측 수행
    results = model(image_path, verbose=False)
    preds = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls.cpu().numpy().item())  # 수정된 부분
            conf = float(box.conf.cpu().numpy().item())  # 수정된 부분
            xyxy = box.xyxy.cpu().numpy().flatten()
            preds.append({
                'bbox': xyxy.tolist(),  # [x1, y1, x2, y2]
                'class': cls,
                'score': conf
            })

    # 예측과 실제 객체 매칭
    TP, FP, FN = match_predictions(preds, targets, iou_threshold=0.5)

    # 메트릭 업데이트
    for cls in TP:
        metrics[cls]['TP'] += TP[cls]
    for cls in FP:
        metrics[cls]['FP'] += FP[cls]
    for cls in FN:
        metrics[cls]['FN'] += FN[cls]

# 클래스별 F1 Score 및 Macro F1 Score 계산
f1_scores = {}
for cls in metrics:
    TP = metrics[cls]['TP']
    FP = metrics[cls]['FP']
    FN = metrics[cls]['FN']
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    f1_scores[cls] = f1
    print(f"클래스 {cls} ({class_names[cls]}): Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

# Macro F1 Score 계산
if f1_scores:
    macro_f1 = np.mean(list(f1_scores.values()))
    print(f"\nMacro F1 Score: {macro_f1:.4f}")
else:
    print("클래스별 F1 Score를 계산할 데이터가 없습니다.")


In [ ]:
import numpy as np

# 클래스별 정밀도, 재현율, F1 점수 입력
class_precision = [0.9568, 0.9808, 0.9807, 0.9893, 0.9924, 0.9984, 0.9659, 0.9846, 0.9738, 0.9929]
class_recall = [1.0000, 0.8667, 0.9643, 1.0000, 1.0000, 1.0000, 0.9231, 0.9935, 0.9545, 0.9941]

# F1 점수를 계산하는 함수
def calculate_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# 클래스별 F1 점수 계산
class_f1 = [calculate_f1(p, r) for p, r in zip(class_precision, class_recall)]

# Macro average 계산
macro_avg_precision = np.mean(class_precision)
macro_avg_recall = np.mean(class_recall)
macro_avg_f1 = np.mean(class_f1)

# 결과 출력
print("Macro Average Precision: {:.4f}".format(macro_avg_precision))
print("Macro Average Recall: {:.4f}".format(macro_avg_recall))
print("Macro Average F1 Score: {:.4f}".format(macro_avg_f1))
